# **NOTEBOOK 1**

En este notebook encontraras algunos de los metodos para imputar valores dentro de un DataFrame, así como los primeros paso para un EDA, adicional de ver algunas maneras de como normalizar o estandarizar los valores dentro de un dataset, com ultimo se muestran algunas formas para tratar la multidimensionalidad y reducir esta.

Recuerde que esto solo es un Notebook que no contiene el fundamento teorico, muchas de la información fue adquirida del libro: "Estadistica para Ciencia de Datos con R y Python"

In [ ]:
# IMPORTACION DE LIBRERIAS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

# **ANALISIS DE DATOS**

In [ ]:
df = pd.DataFrame() #Hacer un dataframe

df.var() #Muestra las varianzas de los valores numericos del df (hay que seleccionar los valores numericos)

df = df.drop('Column1', axis=1) # Borra una columna

df.describe(exclude[object]) #Describe un dataframe sin los objetos

df2 = df.copy() #Realiza una copia del dataframe

df['Column1'].value_counts() #Lista las variables con el numero de valores contenidos en el df

df['Column1'].isnull() #Muestra si tiene valores nulos en dicha columna

df['Column1'].duplicated() #Observar valores duplicados

#rellenar campos de un data frame con valores aleatorios
df['Pais'] = np.random.choise(['Alemania', 'Brasil', 'Japon'], len(df), p =[0.3, 0.3, 0.4])

#JUNTAR DOS DATA FRAMES
df = pd.concat([df1, df2], axis=1)


In [ ]:
# CAMBIAR LOS TIPOS DE DATOS DE UN DF
df = df.astype({
    'Column1':'float64',
    'Column2':'int64'
    'Column3':'String'
})

In [ ]:
# Toma todas las columnas de valores categoricos
categorical = [column_name for column_name in df.columns if df[column_name].dtyoe = "O"]
# Suma los valores nulos de cada columna
df[categorical].isnull().sum()

# Toma todas las columnas de valores que no son categoricos
categorical = [column_name for column_name in df.columns if df[column_name].dtyoe != "O"]
# Suma los valores nulos de cada columna no categorica
df[categorical].isnull().sum()

# **IMPUTACION DE VALORES**
**Rellenar valores nulos, faltantes, o eliminar registros**

In [ ]:
#Rellena los valores nulos con los valores de su media
df = df.fillna(df.mean())

NameError: name 'df' is not defined

In [ ]:
# VALIDAR VALORES DUPLICADOS
def validar_duplicados(df):
    # Comprueba si hay duplicados en el DataFrame
    duplicated = df.duplicated()
    if duplicated.any():
        print("Se encontraron duplicados:")
        print(df[duplicated])
    else:
        print("No se encontraron duplicados.")
validar_duplicados(df)

In [ ]:
# Verificamos la completitud de los valores
completitud = pd.DataFrame(df.isnull().sum())  # Contamos los valores nulos por columna
completitud.reset_index(inplace=True)  # Restablecemos el índice para convertirlo en columna
completitud = completitud.rename(columns={"index": "columna", 0: "total"})  # Renombramos las columnas
completitud["completitud"] = (1 - completitud["total"] / df.shape[0]) * 100  # Calculamos el porcentaje de completitud
completitud = completitud.sort_values(by="completitud", ascending=True)  # Ordenamos por completitud (de menor a mayor)
completitud.reset_index(drop=True, inplace=True)  # Restablecemos el índice después de ordenar

In [ ]:
#BORRAR VARIABLES CON MENOS DEL 80% DE COMPLETITUD
null_cols = list(completitud[completitud["completitud"] < 80]["columna"].values)  # Filtramos las columnas con < 80% de completitud
df = df.drop(columns=null_cols)  # Eliminamos las columnas seleccionadas

# Aseguramos que no tenemos variables con <80% de completitud
completitud = pd.DataFrame(df.isnull().sum())  # Calculamos los valores nulos por columna
completitud.reset_index(inplace=True)  # Restablecemos el índice para convertirlo en columna
completitud = completitud.rename(columns={"index": "columna", 0: "total"})  # Renombramos las columnas
completitud["completitud"] = (1 - completitud["total"] / df.shape[0]) * 100  # Calculamos el porcentaje de completitud
completitud = completitud.sort_values(by="completitud", ascending=True)  # Ordenamos por completitud (de menor a mayor)
completitud.reset_index(drop=True, inplace=True)  # Restablecemos el índice después de ordenar

In [ ]:
# Para tomar los outliers y meterlos en el box
outliers = ['Column1', 'Column2', 'Column3']  # Lista de las columnas a evaluar por outliers
for feature in outliers:
    q1 = df[feature].quantile(0.25)  # Cuartil 1 (25%)
    q3 = df[feature].quantile(0.75)  # Cuartil 3 (75%)
    IQR = q3 - q1  # Rango intercuartil (IQR)
    lower_limit = q1 - (IQR * 1.5)  # Límite inferior para detectar outliers
    upper_limit = q3 + (IQR * 1.5)  # Límite superior para detectar outliers

    # Reemplazar los valores por los límites
    df.loc[df[feature] < lower_limit, feature] = lower_limit  # Valores menores que el límite inferior
    df.loc[df[feature] > upper_limit, feature] = upper_limit  # Valores mayores que el límite superior

In [ ]:
# Rellenar los valores nulos con valores promedios
numerical_features_with_null = [feature for feature in numerical_features if df[feature].isnull().sum()]  # Identificar columnas numéricas con valores nulos

for feature in numerical_features_with_null:
    mean_value = df[feature].mean()  # Calcular el valor promedio de la columna
    df[feature].fillna(mean_value, inplace=True)  # Rellenar los valores nulos con el valor promedio

In [ ]:
# Imputación de valores faltantes
from sklearn.impute import SimpleImputer
import numpy as np

# Identificar las columnas con valores faltantes (no 100% completas)
imputar = list(completitud[completitud["completitud"] != 100]["columna"].values)

# Crear una instancia de SimpleImputer para imputar los valores faltantes con el valor más frecuente
imp = SimpleImputer(missing_values=np.nan, strategy="most_frequent")

# Ajustamos el imputador a las columnas con valores faltantes
imp.fit(df[imputar])

# Aplicamos la imputación de los valores faltantes
df[imputar] = imp.transform(df[imputar])

In [ ]:
df[imputar]=imp.transform(df[imputar]) #Aderimos los valores faltantes

In [ ]:
# Verificamos la completitud de los valores en cada una de las columnas del DataFrame
completitud = pd.DataFrame(df.isnull().sum())  # Contamos los valores nulos por columna
completitud.reset_index(inplace=True)  # Restablecemos el índice para convertirlo en columna
completitud = completitud.rename(columns={"index": "columna", 0: "total"})  # Renombramos las columnas: 'columna' y 'total'
completitud["completitud"] = (1 - completitud["total"] / df.shape[0]) * 100  # Calculamos el porcentaje de completitud (porcentaje de no nulos)
completitud = completitud.sort_values(by="completitud", ascending=True)  # Ordenamos el DataFrame por la columna 'completitud' (de menor a mayor)
completitud.reset_index(drop=True, inplace=True)  # Restablecemos el índice después de ordenar, para numerar las filas de forma secuencial
completitud  # Mostramos el DataFrame con la completitud de cada columna

# **ESCALADORES**

In [ ]:
#NORMALIZAR
from sklearn.preprocessing import Normalizer

norm = Normalizer()
df_norm = norm.fit_transform(df) # esto es un array
df_norm = pd.DataFrame(df_norm, columns=df.columns) #al array lo vuelvo DF

In [ ]:
# ESCALADORES

# Standard Scaler
escalador = StandardScaler()
transformado = escalador.fit_transform(df) # Tomar valores numericos
transformado

#MINMAX Escaler
escalador = MinMaxScaler()
transformado= escalador.fit_transform(df)
transformado

# **TRANSFORMACION DE VALORES CATEGORICOS**

In [ ]:
# TRANSFORMADORES DE VARIABLES CATEGORICAS
# Pone una etiqueta númerica a una variable categorica

# LABEL ENCONDER
from sklearn.processing import LabelEncoder, OneHotEncoder

#Se instancian al crear un objeto con la clase LabelEncoder
Le_En = LabelEncoder()

#Lo usual es que se muestre los valores unicos de una variable
df['Column1'].nunique() #numero de valores unicos
df['Column1'].unique() #lista de valores unicos

df['Column1'] = Le_En.fit_transform(df['Column1'])
#Muestra las clases del label encoder
print(Le_En.classes_)

#Muestra como lista las clases
for i in range(len(Le_En.classes_)):
  print(f'{i} - {Le_En.classes_{i}}')

df['Column1'] = Le_En.inverse_transform(df['Column1']) #Transformación inversa

#ONEHOTENCODER

from sklearn.preprocessing import OneHotEncoder
#Se instancia el transformador
transformador = OneHotEncoder()

enc_df = pd.DataFrame(transformador.fit_transform(df[['sex']]).toarray())

df = df.join(enc_df) #unimos los dos dataframes enc_df y df

# Dummies
dummies = pd.get_dummies(df, columns=["sex"], prefix=["sex"] )
dummies[['sex_male','sex_female']]
df

SyntaxError: f-string: invalid syntax. Perhaps you forgot a comma? (<ipython-input-2-02a7e56ccfe8>, line 20)

#**REDUCCION DE DIMENSIONES**


**CORRELACION**

In [ ]:
#REALIZA LA MATRIZ DE COVARIANZA
df_cov = df.cov()
#Graficamos la matriz de covarianza
sns.heatmap(df.cov)


**VIF FACTOR DE INFLACION DE LA VARIANZA**

In [ ]:
# FACTOR DE INFLACION DE LA VARIANZA VIF
import statsmodels.api as sm  # Importamos statsmodels para trabajar con modelos estadísticos
from statsmodels.stats.outliers_influence import variance_inflation_factor  # Importamos la función para calcular el VIF

# Calculamos el VIF
vif = sm.add_constant(df)  # Añadimos una constante (columna de unos) al DataFrame para incluir el intercepto en el modelo
vif = [variance_inflation_factor(vif.values, i) for i in range(vif.shape[1])]  # Calculamos el VIF para cada columna (inclusive la constante)
vif = pd.Series(vif, index = sm.add_constant(df).columns)  # Convertimos los resultados en una serie, con los nombres de las columnas como índice
vif  # Mostramos el VIF para cada variable

**PCA**

In [ ]:
#PCA
from sklearn.decomposition import PCA
#Se puede añadir ruido
pca = PCA() #opcional se puede agregar un argumento numerico para elegir el numero de variables que se deseas.
pca.fit_transform(df)

#Acumulacion de varianzas y grafico para elegir las variables

ar_var = pca.explained_variance_ratio_.cumsum()
ar_pca_sum = np.cumsum(ar_var)

#Grafico
plt.plot(ar_pca_sum)
plt.xlabel('N componentes')
plt.ylabel('Suma acumulada de varianzas')
plt.show()

#Elegir la cantidad de variables
np.where((ar_pca_sum > 0.8) & (ar_pca_sum < 0.9))
pca_variance = pca.explained_variance_ #explicara cada una de las varianzas de cada variable
pca_variance